In [11]:
import cv2 
from ultralytics import YOLO 
from collections import defaultdict 
import torch
from collections import defaultdict

In [12]:
model =YOLO('yolo11l.pt') #loading yolo11 large pre-trained model

In [13]:
class_list = model.names
print(class_list)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [14]:
cap = cv2.VideoCapture('test_videos/suitcase.mp4') 

In [15]:
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
video_writer = cv2.VideoWriter("suitcase_output.mp4", fourcc, fps, (frame_width, frame_height))

line_y_red = 430  # Red line position

class_counts = defaultdict(int)

crossed_ids = set()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

   
    results = model.track(frame, persist=True, classes=[28])

    if results[0].boxes.data is not None:
       
        boxes = results[0].boxes.xyxy.cpu()
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()
        else:
            track_ids = [-1] * len(boxes)  # Assign dummy ID (-1) if missing

        # Draw red line
        cv2.line(frame, (690, line_y_red), (1130, line_y_red), (0, 0, 255), 3)

       
        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            if conf < 0.8:  # Ignore detections with confidence below 0.8
                continue

            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2 
            cy = (y1 + y2) // 2            

            class_name = class_list[class_idx]

            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) 

           
            if cy > line_y_red and track_id != -1 and track_id not in crossed_ids:
                crossed_ids.add(track_id)
                class_counts[class_name] += 1

        
        y_offset = 30
        for class_name, count in class_counts.items():
            cv2.putText(frame, f"{class_name}: {count}", (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            y_offset += 30

   
    video_writer.write(frame)


cap.release()
video_writer.release()
cv2.destroyAllWindows()


0: 384x640 2 suitcases, 710.8ms
Speed: 3.7ms preprocess, 710.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 suitcases, 726.5ms
Speed: 6.5ms preprocess, 726.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 suitcases, 696.1ms
Speed: 3.5ms preprocess, 696.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 suitcases, 725.2ms
Speed: 3.3ms preprocess, 725.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 suitcases, 717.0ms
Speed: 0.0ms preprocess, 717.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 suitcases, 707.9ms
Speed: 3.0ms preprocess, 707.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 suitcases, 713.4ms
Speed: 3.0ms preprocess, 713.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 suitcases, 709.0ms
Speed: 6.9ms preprocess, 709.0ms inference, 5.8ms 